RunnableParallel

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [4]:
import os

In [5]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

In [6]:
chat = ChatGoogleGenerativeAI(
    google_api_key = os.getenv("GEMINI_API_KEY"),  
    model = "gemini-2.0-flash",
    temperature = 0,
    max_output_tokens = 100,
    model_kwargs = {"seed": 365}
)

In [7]:
string_parser = StrOutputParser()

In [8]:
chain_books = chat_template_books | chat | string_parser
chain_projects = chat_template_projects | chat | string_parser

In [9]:
chain_parallel= RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [10]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '*   Fluent Python\n*   Effective Python: 90 Specific Ways to Write Better Python\n*   Python Cookbook',
 'projects': '*   **Web Scraper with Data Analysis and Visualization**\n*   **Personal Finance Tracker with GUI**\n*   **Simple Game Engine with Custom Game**'}

In [11]:
chain_parallel.get_graph().print_ascii()

                   +-------------------------------+                     
                   | Parallel<books,projects>Input |                     
                   +-------------------------------+                     
                       ****                  ****                        
                   ****                          ****                    
                 **                                  **                  
  +--------------------+                       +--------------------+    
  | ChatPromptTemplate |                       | ChatPromptTemplate |    
  +--------------------+                       +--------------------+    
             *                                            *              
             *                                            *              
             *                                            *              
+------------------------+                   +------------------------+  
| ChatGoogleGenerativeAI |            

In [12]:
%%time 
chain_books.invoke({'programming language':'Python'})

CPU times: total: 0 ns
Wall time: 984 ms


'*   Fluent Python\n*   Effective Python: 90 Specific Ways to Write Better Python\n*   Python Cookbook'

In [13]:
%%time 
chain_projects.invoke({'programming language':'Python'})

CPU times: total: 0 ns
Wall time: 1.4 s


'*   **Web Scraper with Data Analysis and Visualization**\n*   **Personal Finance Tracker with GUI**\n*   **Simple Game Engine with Custom Levels**'

In [14]:
%%time 
chain_parallel.invoke({'programming language':'Python'})

CPU times: total: 31.2 ms
Wall time: 2.33 s


{'books': '*   Fluent Python\n*   Effective Python: 90 Specific Ways to Write Better Python\n*   Python Cookbook',
 'projects': '*   **Web Scraper with Data Analysis and Visualization**\n*   **Personal Finance Tracker with GUI**\n*   **Simple Game Engine with Custom Levels**'}